# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats # used to calculate percentile scores
import xlsxwriter

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.head()
from secret import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://api.iex.cloud/v1/data/core/advanced_stats/{symbol}?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

[{'beta': 1.1083283303635056, 'totalCash': 61555000000, 'currentDebt': 145308000000, 'revenue': 383285000000, 'grossProfit': 169148000000, 'totalRevenue': 383285000000, 'EBITDA': 125820000000, 'revenuePerShare': 24.64, 'revenuePerEmployee': 2607380.95, 'debtToEquity': 5.673462491552152, 'profitMargin': 0.2530623426432028, 'enterpriseValue': 2878734116320, 'enterpriseValueToRevenue': 7.51, 'priceToSales': 7.38, 'priceToBook': 45.53, 'forwardPERatio': 27.58394770420291, 'pegRatio': 85.51071544860974, 'peHigh': 13.32429137049052, 'peLow': 9.976568521032243, 'week52highDate': '2023-12-14', 'week52lowDate': '2023-01-06', 'putCallRatio': 0.4787638496632631, 'companyName': 'Apple Inc', 'marketcap': 2829201116320, 'week52high': 199.62, 'week52low': 123.87, 'week52highSplitAdjustOnly': 199.62, 'week52highDateSplitAdjustOnly': '2023-12-14', 'week52lowSplitAdjustOnly': 124.89, 'week52lowDateSplitAdjustOnly': '2023-01-06', 'week52change': 0.4670859339275013, 'sharesOutstanding': 15552752000, 'floa

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
data[0]['peRatio']

29.16852534996649

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [12]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings[:]:
    
    # the first API call is for parsing company symbol and latestPrice
    batch_api_call_url1 = f'https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data1 = requests.get(batch_api_call_url1).json()
    
    # the second API call is for parsing company year1ChangePercent
    batch_api_call_url2 = f'https://api.iex.cloud/v1/data/core/advanced_stats/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data2 = requests.get(batch_api_call_url2).json()
    
    for i in range(len(data1)):
        # Extract symbol and price from the first API response
        symbol = data1[i]['symbol']
        price = data1[i]['latestPrice']
        
        # Extract year one change percent from the second API response
        pe_ratio = data2[i]['peRatio']
        
        # Create a new row with the combined information
        row_to_add = pd.DataFrame([[symbol, price, pe_ratio, 'N/A']], columns=my_columns)
    
        final_dataframe = pd.concat([final_dataframe, row_to_add], ignore_index=True)
        
final_dataframe

/var/folders/ft/tp0qzzj940v6rmm3bb_9k_bw0000gn/T/ipykernel_1151/229089102.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = pd.concat([final_dataframe, row_to_add], ignore_index=True)


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,130.56,30.954466,N/A
1,AAL,13.59,5.326803,N/A
2,AAP,61.52,19.342935,N/A
3,AAPL,181.09,29.168525,N/A
4,ABBV,162.14,44.863847,N/A
...,...,...,...,...
500,YUM,128.34,23.968681,N/A
501,ZBH,119.98,52.961167,N/A
502,ZBRA,252.77,27.940729,N/A
503,ZION,44.08,4.554481,N/A


In [13]:
# export the dataframe for future uses (token expired)
final_dataframe.to_csv('final_dataframe_project3.csv', index=False)

## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [14]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = True, inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] >= 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True, drop = True) # drop will drop the original index
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,FRC,3.510,0.418421,N/A
1,EMR,95.470,4.103270,N/A
2,VLO,129.000,4.142336,N/A
3,ZION,44.080,4.554481,N/A
4,UAL,41.750,4.639932,N/A
5,ALB,135.830,4.799562,N/A
6,GM,35.990,4.895044,N/A
7,MPC,152.500,5.045527,N/A
8,AAL,13.590,5.326803,N/A
9,WU,11.960,5.720495,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [15]:
def portfolio_input():
    global portfolio_size
    while True:
        try:
            portfolio_size = input('Enter the value of your portfolio: ')
            val = float(portfolio_size)
            print(val)
            break  # Break out of the loop if the conversion is successful
        except ValueError:
            print("That's not a number! Please try again:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [16]:
portfolio_input()

Enter the value of your portfolio: 10000000
10000000.0


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [17]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,FRC,3.510,0.418421,56980
1,EMR,95.470,4.103270,2094
2,VLO,129.000,4.142336,1550
3,ZION,44.080,4.554481,4537
4,UAL,41.750,4.639932,4790
5,ALB,135.830,4.799562,1472
6,GM,35.990,4.895044,5557
7,MPC,152.500,5.045527,1311
8,AAL,13.590,5.326803,14716
9,WU,11.960,5.720495,16722


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [49]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])

composite_columns = ['Ticker', 'Price', 'Number of Shares to Buy', 
              'Price-to-Earnings Ratio', 'PE Percentile', 
              'Price-to-Book Ratio', 'PB Percentile', 'Price-to-Sales Ratio', 'PS Percentile', 
              'EV/EBITDA', 'EV/EBITDA Percentile', 'EV/GP', 'EV/GP Percentile', 'RV Score']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [50]:
composite_dataframe = pd.DataFrame(columns = composite_columns)

for symbol_string in symbol_strings[:]:
    
    # the first API call is for parsing company symbol and latestPrice
    batch_api_call_url1 = f'https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data1 = requests.get(batch_api_call_url1).json()
    
    # the second API call is for parsing company year1ChangePercent
    batch_api_call_url2 = f'https://api.iex.cloud/v1/data/core/advanced_stats/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data2 = requests.get(batch_api_call_url2).json()
    
    for i in range(len(data1)):
        # Extract symbol and price from the first API response
        symbol = data1[i]['symbol']
        price = data1[i]['latestPrice']
        
        # Extract year one change percent from the second API response
        pe_ratio = data2[i]['peRatio']
        pb_ratio = data2[i]['priceToBook']
        ps_ratio = data2[i]['priceToSales']
        EV = data2[i]['enterpriseValue']
        EBITDA = data2[i]['EBITDA']
        GP = data2[i]['grossProfit']
        
        
        # Check if EV and EBITDA have valid non-zero numerical values
        if isinstance(EV, (int, float)) and isinstance(EBITDA, (int, float)) and EBITDA != 0:
            ev_over_ebitda = EV / EBITDA
        else:
            ev_over_ebitda = np.nan

        # Check if EV and GP have valid non-zero numerical values
        if isinstance(EV, (int, float)) and isinstance(GP, (int, float)) and GP != 0:
            ev_over_gp = EV / GP
        else:
            ev_over_gp = np.nan
        
        # Create a new row with the combined information
        row_to_add = pd.DataFrame([[symbol, price, 'N/A', pe_ratio,
                                   'N/A', pb_ratio, 'N/A', ps_ratio, 'N/A',
                                   ev_over_ebitda, 'N/A', ev_over_gp, 'N/A', 'N/A']], columns=composite_columns)
    
        composite_dataframe = pd.concat([composite_dataframe, row_to_add], ignore_index=True)
        
composite_dataframe

/var/folders/ft/tp0qzzj940v6rmm3bb_9k_bw0000gn/T/ipykernel_1151/2697669582.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  composite_dataframe = pd.concat([composite_dataframe, row_to_add], ignore_index=True)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,130.56,N/A,30.954466,N/A,6.57,N/A,5.6200,N/A,24.385279,N/A,11.407947,N/A,N/A
1,AAL,13.60,N/A,5.326803,N/A,-1.67,N/A,0.1617,N/A,4.714798,N/A,0.594585,N/A,N/A
2,AAP,61.52,N/A,19.342935,N/A,1.36,N/A,0.3186,N/A,8.223413,N/A,1.079240,N/A,N/A
3,AAPL,181.18,N/A,29.168525,N/A,45.53,N/A,7.3800,N/A,22.879782,N/A,17.019025,N/A,N/A
4,ABBV,162.14,N/A,44.863847,N/A,24.03,N/A,5.2700,N/A,13.447516,N/A,9.324655,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,128.34,N/A,23.968681,N/A,-4.40,N/A,5.1100,N/A,19.542825,N/A,8.866994,N/A,N/A
501,ZBH,119.98,N/A,52.961167,N/A,2.01,N/A,3.4500,N/A,15.811049,N/A,5.813042,N/A,N/A
502,ZBRA,252.69,N/A,27.940729,N/A,4.31,N/A,2.5600,N/A,17.436283,N/A,6.442559,N/A,N/A
503,ZION,44.05,N/A,4.554481,N/A,1.30,N/A,1.2700,N/A,3.478560,N/A,1.312773,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [51]:
composite_dataframe[composite_dataframe.isnull().any(axis = 1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
6,ABMD,381.020,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
28,ALXN,182.500,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,289.590,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
48,ATVI,94.420,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,94.920,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
121,CTXS,103.900,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,24.420,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
137,DISH,5.770,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,49.260,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [52]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    # use shift tab to get arguments for function
    composite_dataframe[column].fillna(composite_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [53]:
composite_dataframe[composite_dataframe.isnull().any(axis = 1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [76]:
metrics = {
        'Price-to-Earnings Ratio': 'PE Percentile', 
        'Price-to-Book Ratio': 'PB Percentile',
        'Price-to-Sales Ratio': 'PS Percentile',
        'EV/EBITDA': 'EV/EBITDA Percentile', 
        'EV/GP': 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in composite_dataframe.index:
        percentile = stats.percentileofscore(composite_dataframe[metric], composite_dataframe.loc[row, metric])/100
        composite_dataframe.loc[row, metrics[metric]] = percentile

In [77]:
composite_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,3.51,14245,0.418421,0.16,0.0456,0.08,0.0934,0.02,0.270279,0.04,0.094878,0.02,4.158416
1,AAL,13.60,3676,5.326803,0.28,-1.6700,0.06,0.1617,0.06,4.714798,0.32,0.594585,0.1,5.623762
2,GM,35.99,1389,4.895044,0.24,0.6526,0.12,0.2826,0.14,1.303717,0.06,0.920818,0.2,5.663366
3,WBA,25.01,1999,-6.806940,0.12,1.0500,0.4,0.1507,0.04,-6.378037,0.02,1.081618,0.34,5.782178
4,UAL,41.76,1197,4.639932,0.22,1.5000,0.78,0.2529,0.1,3.216140,0.12,0.497454,0.08,7.841584
5,F,11.85,4219,7.453255,0.4,1.0400,0.38,0.2636,0.12,2.340069,0.08,1.373183,0.62,8.138614
6,UNM,45.84,1090,7.306301,0.36,0.9365,0.28,0.7355,0.58,4.346059,0.24,0.735492,0.18,8.475248
7,C,54.33,920,8.367301,0.54,0.5416,0.1,0.6965,0.52,4.536453,0.3,0.733229,0.16,8.594059
8,KSS,26.44,1891,-20.676659,0.02,0.7827,0.18,0.1674,0.08,9.018321,0.98,1.226279,0.5,9.306931
9,SYF,38.82,1287,6.637890,0.34,1.1900,0.58,0.7248,0.56,4.367870,0.26,0.967577,0.22,9.623762


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [78]:
from statistics import mean

value_percentiles = ['PE Percentile', 
        'PB Percentile',
        'PS Percentile',
        'EV/EBITDA Percentile', 
        'EV/GP Percentile']

for row in composite_dataframe.index:
    rv_percentiles = []
    for percentile in value_percentiles:
        rv_percentiles.append(composite_dataframe.loc[row, percentile])
    composite_dataframe.loc[row, 'RV Score'] = mean(rv_percentiles)
composite_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,3.51,14245,0.418421,0.16,0.0456,0.08,0.0934,0.02,0.270279,0.04,0.094878,0.02,0.064
1,AAL,13.60,3676,5.326803,0.28,-1.6700,0.06,0.1617,0.06,4.714798,0.32,0.594585,0.1,0.164
2,GM,35.99,1389,4.895044,0.24,0.6526,0.12,0.2826,0.14,1.303717,0.06,0.920818,0.2,0.152
3,WBA,25.01,1999,-6.806940,0.12,1.0500,0.4,0.1507,0.04,-6.378037,0.02,1.081618,0.34,0.184
4,UAL,41.76,1197,4.639932,0.22,1.5000,0.78,0.2529,0.1,3.216140,0.12,0.497454,0.08,0.26
5,F,11.85,4219,7.453255,0.4,1.0400,0.38,0.2636,0.12,2.340069,0.08,1.373183,0.62,0.32
6,UNM,45.84,1090,7.306301,0.36,0.9365,0.28,0.7355,0.58,4.346059,0.24,0.735492,0.18,0.328
7,C,54.33,920,8.367301,0.54,0.5416,0.1,0.6965,0.52,4.536453,0.3,0.733229,0.16,0.324
8,KSS,26.44,1891,-20.676659,0.02,0.7827,0.18,0.1674,0.08,9.018321,0.98,1.226279,0.5,0.352
9,SYF,38.82,1287,6.637890,0.34,1.1900,0.58,0.7248,0.56,4.367870,0.26,0.967577,0.22,0.392


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [79]:
composite_dataframe.sort_values('RV Score', ascending = True, inplace = True)
composite_dataframe = composite_dataframe[:50]
composite_dataframe.reset_index(drop = True, inplace = True)
composite_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,3.51,14245,0.418421,0.16,0.0456,0.08,0.0934,0.02,0.270279,0.04,0.094878,0.02,0.064
1,GM,35.99,1389,4.895044,0.24,0.6526,0.12,0.2826,0.14,1.303717,0.06,0.920818,0.2,0.152
2,AAL,13.60,3676,5.326803,0.28,-1.6700,0.06,0.1617,0.06,4.714798,0.32,0.594585,0.1,0.164
3,WBA,25.01,1999,-6.806940,0.12,1.0500,0.4,0.1507,0.04,-6.378037,0.02,1.081618,0.34,0.184
4,UAL,41.76,1197,4.639932,0.22,1.5000,0.78,0.2529,0.1,3.216140,0.12,0.497454,0.08,0.26
5,F,11.85,4219,7.453255,0.4,1.0400,0.38,0.2636,0.12,2.340069,0.08,1.373183,0.62,0.32
6,C,54.33,920,8.367301,0.54,0.5416,0.1,0.6965,0.52,4.536453,0.3,0.733229,0.16,0.324
7,UNM,45.84,1090,7.306301,0.36,0.9365,0.28,0.7355,0.58,4.346059,0.24,0.735492,0.18,0.328
8,KSS,26.44,1891,-20.676659,0.02,0.7827,0.18,0.1674,0.08,9.018321,0.98,1.226279,0.5,0.352
9,SYF,38.82,1287,6.637890,0.34,1.1900,0.58,0.7248,0.56,4.367870,0.26,0.967577,0.22,0.392


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [80]:
portfolio_input()

Enter the value of your portfolio: 
That's not a number! Please try again:
Enter the value of your portfolio: 2500000
2500000.0


In [81]:
position_size = float(portfolio_size)/len(composite_dataframe.index)
for i in range(0, len(composite_dataframe.index)):
    composite_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/composite_dataframe.loc[i, 'Price'])
composite_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,3.51,14245,0.418421,0.16,0.0456,0.08,0.0934,0.02,0.270279,0.04,0.094878,0.02,0.064
1,GM,35.99,1389,4.895044,0.24,0.6526,0.12,0.2826,0.14,1.303717,0.06,0.920818,0.2,0.152
2,AAL,13.60,3676,5.326803,0.28,-1.6700,0.06,0.1617,0.06,4.714798,0.32,0.594585,0.1,0.164
3,WBA,25.01,1999,-6.806940,0.12,1.0500,0.4,0.1507,0.04,-6.378037,0.02,1.081618,0.34,0.184
4,UAL,41.76,1197,4.639932,0.22,1.5000,0.78,0.2529,0.1,3.216140,0.12,0.497454,0.08,0.26
5,F,11.85,4219,7.453255,0.4,1.0400,0.38,0.2636,0.12,2.340069,0.08,1.373183,0.62,0.32
6,C,54.33,920,8.367301,0.54,0.5416,0.1,0.6965,0.52,4.536453,0.3,0.733229,0.16,0.324
7,UNM,45.84,1090,7.306301,0.36,0.9365,0.28,0.7355,0.58,4.346059,0.24,0.735492,0.18,0.328
8,KSS,26.44,1891,-20.676659,0.02,0.7827,0.18,0.1674,0.08,9.018321,0.98,1.226279,0.5,0.352
9,SYF,38.82,1287,6.637890,0.34,1.1900,0.58,0.7248,0.56,4.367870,0.26,0.967577,0.22,0.392


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [82]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
composite_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [83]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.000',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

title_format = writer.book.add_format({
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1,
    'bold': True  # This will make the font bold
})

In [84]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-BOok Ratio', float_template],
                    'G': ['PB Ratio', percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                    }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], title_format)

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [85]:
writer.close()